In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run parse.ipynb

In [ ]:
# alright, let's try with one movement, one performer!
X = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))
y = torch.Tensor(parseGroundtruthFile('../bach-violin-dataset/dataset/ground-truth/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))

In [ ]:
# never mind, let's try combining all the inputs.....?

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.actfn = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.actfn(x)
        x = self.linear2(x)
        return x

In [ ]:
model = MLP(3, 4, 2)

In [ ]:
# Just in case our model is retaining bad parameters
# model.reset_parameters()

In [ ]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [ ]:
# Template code from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#The-Basics-of-PyTorch
def train_model(model, optimizer, input_notes, truth, loss_module, num_epochs=100):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        # data_inputs = data_inputs.to(device)
        # data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)
        print(preds)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, y)
        print(loss)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [ ]:
train_model(model, optimizer, X, y, loss)

In [ ]:
X_test = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov3.csv'))
y_test = torch.Tensor.tolist(model(X_test))

X_test = torch.Tensor.tolist(X_test)
print(X_test)
for i in range(len(y_test)):
    y_test[i].append(X_test[i][2])

print(y_test)